In [11]:
print('訓練之前載入套件')
print('1-X與y資料準備')
print('2-資料切割')
from sklearn.model_selection import train_test_split
print('3-產生模型物件：各種模型')
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
print('3-整體學習')
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

訓練之前載入套件
1-X與y資料準備
2-資料切割
3-產生模型物件：各種模型
3-整體學習


In [12]:
print('建立起模型清單1：KNN')
clf_list = [('KNN',KNeighborsClassifier()),
            ('DecisionTree1',DecisionTreeClassifier()),
            ('DecisionTree2',DecisionTreeClassifier(max_depth=3)),
            ('SVC_linear',SVC(kernel='linear',C=0.5,gamma='scale',probability=True)),
            ('SVC_rbf',SVC(kernel='rbf',C=0.5,gamma='scale',probability=True)),
            ('GaussianNB',GaussianNB()),
            ('LogisticRegression',LogisticRegression(solver='liblinear',max_iter=100)),
            ('MultinomialNB',MultinomialNB())]

print('建立起模型清單2：Voting Classifier')
clf_list2 = [('hard vote',VotingClassifier(clf_list, voting='hard')),
             ('soft vote',VotingClassifier(clf_list, voting='soft'))]

建立起模型清單1：KNN
建立起模型清單2：Voting Classifier


In [13]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import joblib

In [ ]:
df = pd.read_csv("model_files/ltn1_sentiment1.csv").dropna(subset=["title", "class"])
# 使用 jieba 進行中文斷詞，並將斷詞結果以空格(" ")連接
df["cut_title"] = df["title"].apply(lambda x: " ".join(jieba.cut(str(x))))
le = LabelEncoder()  # 初始化 LabelEncoder (類別標籤編碼器)：將 "class" 欄位中的類別標籤轉換為數字
df["label"] = le.fit_transform(df["class"])
    # fit() 方法用於擬合資料，即學習 "class" 欄位中每個不同的類別標籤。
    # transform() 方法則將 "class" 欄位中的每個類別標籤轉換為對應的數字。
# joblib.dump(le, "label_encoder_nb.joblib")  # 將訓練好的 LabelEncoder 物件儲存到檔案
    # joblib.dump(le, "label_encoder_nb.joblib")：這是 joblib 庫的用法，用於將 Python 物件序列化到磁碟檔案。
    # 它將訓練好的 LabelEncoder 物件 le 儲存到 ....joblib 檔案中，以便後續可以載入並用於轉換新的類別標籤。
print(df.head(3))
df.to_csv('model_files/ltn1_sen_fit.csv', index=False, encoding='utf-8-sig')

vectorizer = TfidfVectorizer(max_features=5000)  # 初始化 TfidfVectorizer：用於將文本資料轉換為 TF-IDF 特徵向量的工具
X = vectorizer.fit_transform(df["cut_title"])    # 將斷詞後的 "cut_title" 欄位轉換為 TF-IDF 特徵矩陣
print(X)
y = df["label"]
# joblib.dump(vectorizer, "tfidf_vectorizer.joblib")

                          title class        time  \
1  美國股市受科技股拖累全收黑  台積電ADR小跌0.61％    財經  2024/12/31   
2     金融時報： 機器人市場即將爆發 就像ChatGPT    財經  2024/12/31   
3       台積電今年股價飆漲逾八成 市值年增超過12兆元    財經  2024/12/31   

                                                link sentiment  \
1  https://ec.ltn.com.tw/article/breakingnews/490...        負面   
2  https://ec.ltn.com.tw/article/breakingnews/490...        中立   
3  https://ec.ltn.com.tw/article/breakingnews/490...        中立   

                                   cut_title  label  
1  美國 股市 受 科技股 拖累 全收 黑     台積電 ADR 小跌 0.61 ％      6  
2      金融 時報 ：   機器 人 市場 即將 爆發   就 像 ChatGPT      6  
3         台積電 今年 股價 飆漲 逾 八成   市值 年 增超過 12 兆元      6  
  (0, 2799)	0.2739464783227707
  (0, 2866)	0.34460137929222484
  (0, 2635)	0.33504217654630125
  (0, 1780)	0.354078729929245
  (0, 707)	0.3511876744615682
  (0, 995)	0.2545680095602568
  (0, 139)	0.3360786291435709
  (0, 1440)	0.3657128041230171
  (0, 113)	0.3657128041230171
  (1, 3440)	0.3963891158210885

In [15]:
print('4-模型物件開始進行訓練')
print('5-模型儲存')
print('6-模型載入')
import joblib
print('7-模型進行預估')
print('8-模型進行評估')
from sklearn.metrics import accuracy_score #準確率
from sklearn.metrics import classification_report #分類報告
from sklearn.metrics import confusion_matrix #混淆矩陣
from scipy.sparse import issparse # 檢查是否為稀疏矩陣

4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估


In [16]:
print('各種模型進行訓練與預測')
print('1-X與y資料準備')
print('2-資料切割')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

各種模型進行訓練與預測
1-X與y資料準備
2-資料切割


In [17]:
# 用於 SVC 模型
"""
# 只取出 X 和 y 的前 1000 筆數據
X_subset = X[:1000]
y_subset = y[:1000]

# 使用前 1000 筆數據劃分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.3, random_state=42)

print("X_train 的形狀:", X_train.shape)
print("X_test 的形狀:", X_test.shape)
print("y_train 的形狀:", y_train.shape)
print("y_test 的形狀:", y_test.shape)
"""

'\n# 只取出 X 和 y 的前 1000 筆數據\nX_subset = X[:1000]\ny_subset = y[:1000]\n\n# 使用前 1000 筆數據劃分訓練集和測試集\nX_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.3, random_state=42)\n\nprint("X_train 的形狀:", X_train.shape)\nprint("X_test 的形狀:", X_test.shape)\nprint("y_train 的形狀:", y_train.shape)\nprint("y_test 的形狀:", y_test.shape)\n'

## 1. 

In [18]:
def train_and_evaluate_model(model_tuple, X_train, y_train, X_test, y_test):
    """
    訓練、儲存、載入並評估單個分類模型。
    Args:
        model_tuple (tuple): 包含模型名稱 (str) 和模型物件的元組。
        X_train (array-like): 訓練資料的特徵。 y_train (array-like): 訓練資料的標籤。
        X_test (array-like): 測試資料的特徵。  y_test (array-like): 測試資料的標籤。
    """
    model_name = model_tuple[0]
    model = model_tuple[1]

    print('Model name:', model_name)
    print('3-產生模型物件')

    print('4-模型物件開始進行訓練')
    if issparse(X_train):
        X_train_dense = X_train.toarray()
        model.fit(X_train_dense, y_train)
    else:
        model.fit(X_train, y_train)

    filename = f'{model_name}.joblib'
    print('5-模型儲存')
    joblib.dump(model, filename)
    print('6-模型載入')
    loaded_model = joblib.load(filename)

    print('7-模型進行預估')
    # predict1 = loaded_model.predict(X_test)
    # predict2 = loaded_model.predict(X_train)
    # 將稀疏矩陣轉換為稠密矩陣
    if issparse(X_test):
        X_test_dense = X_test.toarray()
        predict1 = loaded_model.predict(X_test_dense)
    else:
        predict1 = loaded_model.predict(X_test)

    # 訓練集的預估也需要轉換（雖然錯誤通常出現在測試集）
    if issparse(X_train):
        predict2 = loaded_model.predict(X_train.toarray())
    else:
        predict2 = loaded_model.predict(X_train)

    print('8-模型進行評估')
    print('訓練的準確率\n', accuracy_score(y_train, predict2))
    print('測試的準確率\n', accuracy_score(y_test, predict1))
    print('分類報告\n', classification_report(y_test, predict1))
    print('混淆矩陣\n', confusion_matrix(y_test, predict1))
    print('-' * 30) # 添加分隔線以區分不同模型的輸出

# 模型列表
clf_list = [('KNN', KNeighborsClassifier()),
            ('DecisionTree1', DecisionTreeClassifier()),
            ('DecisionTree2', DecisionTreeClassifier(max_depth=3)),
            ('SVC_linear', SVC(kernel='linear', C=0.5, gamma='scale', probability=True)),
            ('SVC_rbf', SVC(kernel='rbf', C=0.5, gamma='scale', probability=True)),
            ('GaussianNB', GaussianNB()),
            ('LogisticRegression', LogisticRegression(solver='liblinear', max_iter=100)),
            ('MultinomialNB', MultinomialNB())]

# for model_info in clf_list:  # 使用 for 迴圈逐次將 clf_list 中的元素作為輸入資料帶入函數
#     train_and_evaluate_model(model_info, X_train, y_train, X_test, y_test)

In [19]:
train_and_evaluate_model(('MultinomialNB', MultinomialNB()), X_train, y_train, X_test, y_test)

Model name: MultinomialNB
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.9974520129098012
測試的準確率
 0.9968295904887715
分類報告
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       172
           1       1.00      1.00      1.00       464
           2       1.00      1.00      1.00       122
           3       1.00      0.99      0.99      1274
           4       1.00      1.00      1.00       124
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00      5046
           7       1.00      1.00      1.00       291
           8       1.00      1.00      1.00        69

    accuracy                           1.00      7570
   macro avg       1.00      1.00      1.00      7570
weighted avg       1.00      1.00      1.00      7570

混淆矩陣
 [[ 172    0    0    0    0    0    0    0    0]
 [   0  464    0    0    0    0    0    0    0]
 [   0    0  122    0    0    0    0    0    0]
 [   

In [16]:
train_and_evaluate_model(('LogisticRegression', LogisticRegression(solver='liblinear', max_iter=100)),
                          X_train, y_train, X_test, y_test)

model name: LogisticRegression
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.996602683879735
測試的準確率
 0.9961690885072655
分類報告
               precision    recall  f1-score   support

           0       1.00      0.97      0.98       196
           1       1.00      0.99      0.99       491
           2       1.00      1.00      1.00       115
           3       1.00      0.99      1.00      1316
           4       1.00      1.00      1.00       140
           5       1.00      0.91      0.95        11
           6       0.99      1.00      1.00      4951
           7       1.00      1.00      1.00       298
           8       1.00      0.92      0.96        52

    accuracy                           1.00      7570
   macro avg       1.00      0.98      0.99      7570
weighted avg       1.00      1.00      1.00      7570

混淆矩陣
 [[ 190    0    0    0    0    0    6    0    0]
 [   0  485    0    0    0    0    6    0    0]
 [   0    0  115    0    0    0    0    0    0]
 

In [15]:
train_and_evaluate_model(('GaussianNB', GaussianNB()), X_train, y_train, X_test, y_test)

model name: GaussianNB
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.994790781948927
測試的準確率
 0.9937912813738441
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       196
           1       0.98      1.00      0.99       491
           2       1.00      1.00      1.00       115
           3       0.98      0.99      0.99      1316
           4       1.00      1.00      1.00       140
           5       1.00      1.00      1.00        11
           6       1.00      0.99      1.00      4951
           7       0.97      0.98      0.97       298
           8       1.00      1.00      1.00        52

    accuracy                           0.99      7570
   macro avg       0.99      1.00      0.99      7570
weighted avg       0.99      0.99      0.99      7570

混淆矩陣
 [[ 196    0    0    0    0    0    0    0    0]
 [   0  491    0    0    0    0    0    0    0]
 [   0    0  115    0    0    0    0    0    0]
 [   0   

In [9]:
train_and_evaluate_model(('SVC_rbf', SVC(kernel='rbf', C=0.5, gamma='scale', probability=True)), 
                         X_train, y_train, X_test, y_test)

Model name: SVC_rbf
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.9814285714285714
測試的準確率
 0.9633333333333334
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      0.75      0.86        24
           2       1.00      0.33      0.50         3
           3       1.00      0.93      0.96        42
           4       1.00      1.00      1.00         4
           6       0.95      1.00      0.97       204
           7       1.00      1.00      1.00        17

    accuracy                           0.96       300
   macro avg       0.99      0.86      0.90       300
weighted avg       0.97      0.96      0.96       300

混淆矩陣
 [[  6   0   0   0   0   0   0]
 [  0  18   0   0   0   6   0]
 [  0   0   1   0   0   2   0]
 [  0   0   0  39   0   3   0]
 [  0   0   0   0   4   0   0]
 [  0   0   0   0   0 204   0]
 [  0   0   0   0   0   0  17]]
------------------------------


In [10]:
train_and_evaluate_model(('SVC_linear', SVC(kernel='linear', C=0.5, gamma='scale', probability=True)), 
                         X_train, y_train, X_test, y_test)

Model name: SVC_linear
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.9857142857142858
測試的準確率
 0.9666666666666667
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      0.75      0.86        24
           2       1.00      0.33      0.50         3
           3       1.00      0.95      0.98        42
           4       1.00      1.00      1.00         4
           6       0.95      1.00      0.98       204
           7       1.00      1.00      1.00        17

    accuracy                           0.97       300
   macro avg       0.99      0.86      0.90       300
weighted avg       0.97      0.97      0.96       300

混淆矩陣
 [[  6   0   0   0   0   0   0]
 [  0  18   0   0   0   6   0]
 [  0   0   1   0   0   2   0]
 [  0   0   0  40   0   2   0]
 [  0   0   0   0   4   0   0]
 [  0   0   0   0   0 204   0]
 [  0   0   0   0   0   0  17]]
------------------------------


In [28]:
train_and_evaluate_model(('DecisionTree2', DecisionTreeClassifier(max_depth=9)), X_train, y_train, X_test, y_test)

Model name: DecisionTree2
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.7685295283392787
測試的準確率
 0.7719947159841479
分類報告
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       212
           1       1.00      0.32      0.49       471
           2       1.00      1.00      1.00       111
           3       0.97      0.20      0.34      1249
           4       0.00      0.00      0.00       119
           5       0.00      0.00      0.00         9
           6       0.74      1.00      0.85      5009
           7       1.00      1.00      1.00       326
           8       0.00      0.00      0.00        64

    accuracy                           0.77      7570
   macro avg       0.52      0.39      0.41      7570
weighted avg       0.77      0.77      0.71      7570

混淆矩陣
 [[   0    0    0    0    0    0  212    0    0]
 [   0  152    0    0    0    0  319    0    0]
 [   0    0  111    0    0    0    0    0    0]
 [   

C:\Users\yjche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yjche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\yjche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

In [29]:
train_and_evaluate_model(('DecisionTree1', DecisionTreeClassifier()), X_train, y_train, X_test, y_test)

Model name: DecisionTree1
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.9983013419398675
測試的準確率
 0.9985468956406869
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       212
           1       1.00      1.00      1.00       471
           2       1.00      1.00      1.00       111
           3       0.99      1.00      1.00      1249
           4       1.00      1.00      1.00       119
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00      5009
           7       1.00      1.00      1.00       326
           8       1.00      1.00      1.00        64

    accuracy                           1.00      7570
   macro avg       1.00      1.00      1.00      7570
weighted avg       1.00      1.00      1.00      7570

混淆矩陣
 [[ 212    0    0    0    0    0    0    0    0]
 [   0  471    0    0    0    0    0    0    0]
 [   0    0  111    0    0    0    0    0    0]
 [   

In [18]:
train_and_evaluate_model(('KNN', KNeighborsClassifier()), X_train, y_train, X_test, y_test)

model name: KNN
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.9982447200045298
測試的準確率
 0.997886393659181
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       196
           1       1.00      1.00      1.00       491
           2       1.00      1.00      1.00       115
           3       0.99      1.00      0.99      1316
           4       1.00      1.00      1.00       140
           5       1.00      0.91      0.95        11
           6       1.00      1.00      1.00      4951
           7       1.00      1.00      1.00       298
           8       1.00      1.00      1.00        52

    accuracy                           1.00      7570
   macro avg       1.00      0.99      0.99      7570
weighted avg       1.00      1.00      1.00      7570

混淆矩陣
 [[ 196    0    0    0    0    0    0    0    0]
 [   0  491    0    0    0    0    0    0    0]
 [   0    0  115    0    0    0    0    0    0]
 [   0    0    0

# 2. 

In [18]:
# 只取出 X 和 y 的前 1000 筆數據
X_subset = X[:1000]
y_subset = y[:1000]

# 使用前 1000 筆數據劃分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.3, random_state=42)

print("X_train 的形狀:", X_train.shape)
print("X_test 的形狀:", X_test.shape)
print("y_train 的形狀:", y_train.shape)
print("y_test 的形狀:", y_test.shape)

X_train 的形狀: (700, 3743)
X_test 的形狀: (300, 3743)
y_train 的形狀: (700,)
y_test 的形狀: (300,)


In [ ]:
import joblib
from scipy.sparse import issparse
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os

def train_and_evaluate_voting_model(model_tuple, X_train, y_train, X_test, y_test):
    """
    訓練、儲存、載入並評估 VotingClassifier 模型。

    Args:
        model_tuple (tuple): 包含模型名稱 (str) 和 VotingClassifier 物件的元組。
        X_train (array-like): 訓練資料的特徵。
        y_train (array-like): 訓練資料的標籤。
        X_test (array-like): 測試資料的特徵。
        y_test (array-like): 測試資料的標籤。
    """
    model_name = model_tuple[0]
    model = model_tuple[1]

    print('model name:', model_name)
    print('3-產生模型物件')

    print('4-模型物件開始進行訓練')
    #model.fit(X_train, y_train)
    if issparse(X_train):
        X_train_dense = X_train.toarray()
        model.fit(X_train_dense, y_train)
    else:
        model.fit(X_train, y_train)

    filename = f'{model_name}.pkl'
    print('5-模型儲存')
    # 設定儲存路徑和檔案名
    output_dir = 'model_files'
    filename = f'{model_name}.pkl'
    filepath = os.path.join(output_dir, filename)
    # 檢查目錄是否存在，如果不存在則創建
    os.makedirs(output_dir, exist_ok=True)

    joblib.dump(model, filepath)  # filename
    print('6-模型載入')
    loaded_model = joblib.load(filename)

    print('7-模型進行預估')
    # predict1 = loaded_model.predict(X_test)
    # predict2 = loaded_model.predict(X_train)
    # 將稀疏矩陣轉換為稠密矩陣
    if issparse(X_test):
        X_test_dense = X_test.toarray()
        predict1 = loaded_model.predict(X_test_dense)
    else:
        predict1 = loaded_model.predict(X_test)

    # 訓練集的預估也需要轉換（雖然錯誤通常出現在測試集）
    if issparse(X_train):
        predict2 = loaded_model.predict(X_train.toarray())
    else:
        predict2 = loaded_model.predict(X_train)

    print('8-模型進行評估')
    print('訓練的準確率\n', accuracy_score(y_train, predict2))
    print('測試的準確率\n', accuracy_score(y_test, predict1))
    print('分類報告\n', classification_report(y_test, predict1))
    print('混淆矩陣\n', confusion_matrix(y_test, predict1))
    print('-' * 30) # 添加分隔線以區分不同模型的輸出

# VotingClassifier 模型列表
clf_list2 = [('hard vote', VotingClassifier(clf_list, voting='hard')),
             ('soft vote', VotingClassifier(clf_list, voting='soft'))]

# 使用 for 迴圈逐次將 clf_list2 中的元素作為輸入資料帶入函數
# for model_info in clf_list2:
#     train_and_evaluate_voting_model(model_info, X_train, y_train, X_test, y_test)

In [20]:
train_and_evaluate_voting_model(('hard vote', VotingClassifier(clf_list, voting='hard')),
                                 X_train, y_train, X_test, y_test)

model name: hard vote
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 0.9828571428571429
測試的準確率
 0.9633333333333334
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      0.75      0.86        24
           2       1.00      0.33      0.50         3
           3       1.00      0.93      0.96        42
           4       1.00      1.00      1.00         4
           6       0.95      1.00      0.97       204
           7       1.00      1.00      1.00        17

    accuracy                           0.96       300
   macro avg       0.99      0.86      0.90       300
weighted avg       0.97      0.96      0.96       300

混淆矩陣
 [[  6   0   0   0   0   0   0]
 [  0  18   0   0   0   6   0]
 [  0   0   1   0   0   2   0]
 [  0   0   0  39   0   3   0]
 [  0   0   0   0   4   0   0]
 [  0   0   0   0   0 204   0]
 [  0   0   0   0   0   0  17]]
------------------------------


In [21]:
train_and_evaluate_voting_model(('soft vote', VotingClassifier(clf_list, voting='soft')),
                                 X_train, y_train, X_test, y_test)

model name: soft vote
3-產生模型物件
4-模型物件開始進行訓練
5-模型儲存
6-模型載入
7-模型進行預估
8-模型進行評估
訓練的準確率
 1.0
測試的準確率
 0.9866666666666667
分類報告
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        24
           2       1.00      0.33      0.50         3
           3       1.00      0.95      0.98        42
           4       1.00      1.00      1.00         4
           6       0.98      1.00      0.99       204
           7       1.00      1.00      1.00        17

    accuracy                           0.99       300
   macro avg       1.00      0.90      0.92       300
weighted avg       0.99      0.99      0.98       300

混淆矩陣
 [[  6   0   0   0   0   0   0]
 [  0  24   0   0   0   0   0]
 [  0   0   1   0   0   2   0]
 [  0   0   0  40   0   2   0]
 [  0   0   0   0   4   0   0]
 [  0   0   0   0   0 204   0]
 [  0   0   0   0   0   0  17]]
------------------------------
